# Time Series Project

En la primera etapa nos ocupamos de cargar los datos

In [1]:
import pandas as pd

import warnings

warnings.filterwarnings("ignore")

from paquete_proyecto.iniciando.bases import importar_databases
from paquete_proyecto.herramientas.data_info import data_info

In [2]:
ventas, ventas_sin_duplicados = importar_databases()

La longitud de la serie Nº 1 es de: 127255
La longitud de la serie Nº 2 es de: 108794


In [3]:
display(ventas.sample(3))
data_info(ventas, "ventas")

,IdCliente,NombreCliente,Empleado,Referencia,Descripcion,CodigoFamilia,Familia,Cantidad,Ventas,Localidad,Sede,Area
Fecha,,,,,,,,,,,,
2018-10-13,C1076622248,BERNAL VIVIANA,36.0,LUA-0016,MOBIL SUPER 2000 10W30 GALON,103.0,LUBRICANTES,1,113000,Engativa,Calle 80,31.0
2017-09-21,C4061315,TELLEZ OSCAR,14.0,SE-CS-1-0001,ALINEACION AUTO,106.0,SERVICIOS,1,16807,Suba,Suba,32.0
2018-08-21,C52112659,TINJACA SANDRA,36.0,SE-CS-1-0001,ALINEACION AUTO,106.0,SERVICIOS,1,33614,Engativa,Calle 80,31.0


info de ventas,columna,type,count,NaN,NaN_pct,unique,unique_pct
0,IdCliente,object,127255,0,0.0 %,24122,18.96 %
1,NombreCliente,object,127255,0,0.0 %,21840,17.16 %
2,Empleado,float64,127255,0,0.0 %,56,0.04 %
3,Referencia,object,127255,0,0.0 %,1854,1.46 %
4,Descripcion,object,127255,0,0.0 %,1814,1.43 %
5,CodigoFamilia,float64,127255,0,0.0 %,5,0.0 %
6,Familia,object,127255,0,0.0 %,5,0.0 %
7,Cantidad,object,127255,0,0.0 %,213,0.17 %
8,Ventas,object,127255,0,0.0 %,14279,11.22 %
9,Localidad,object,127255,0,0.0 %,6,0.0 %


<div>
    <h4>Evaluación de cada columna</h4>
    <ol>
        <li> <b>IdCliente</b>: object - código alfanumérico</li><br />
        <li> <b>NombreCliente</b>: object - string = Nombre, Apellido</li><br />
        <li> <b>>> Empleado</b>: float - cambiar a int</li><br />
        <li> <b>Referencia</b>: object: categoría ordinal</li><br />
        <li> <b>Descripcion</b>: object: categoria cualitativa </li><br />
        <li> <b>>> CodigoFamilia</b>: float64: Cambiar a int</li><br />
        <li> <b>Familia</b>: object: categoria cualitativa</li><br />
        <li> <b>>>Cantidad</b>: object: cambiar a int</li><br />
        <li> <b>>> Ventas</b>: int: cambiar a float</li><br />
        <li> <b>Localidad</b>: object: categoria bussines</li><br />
        <li> <b>>> Area</b>: float: cambiar a int</li><br />
    </ol>
</div>

In [4]:
from paquete_proyecto.iniciando.variables import ajustar_tipos

In [5]:
ajustar_tipos(ventas)

,IdCliente,NombreCliente,Empleado,Referencia,Descripcion,CodigoFamilia,Familia,Cantidad,Ventas,Localidad,Sede,Area
Fecha,,,,,,,,,,,,
2016-01-09,C1070956433,PINTO JOHN,24,LL-2R-0038,PILOT STREET 110/80-17,101,LLANTA,1,129310.0,Puente aranda,Calle 13,35
2016-01-09,C19347046,RUIZ LUIS FERNANDO,10,SE-CS-2-0001,BALANCEO AUTO,106,SERVICIOS,4,31035.0,Engativa,Calle 80,31
2016-01-09,C800232717,CORSO Y CIA S EN C,10,SE-CS-2-0002,BALANCEO CAMIONETA,106,SERVICIOS,4,31034.0,Engativa,Calle 80,31
2016-01-09,C79523685,GONZALEZ JUAN CARLOS,10,SE-CS-2-0002,BALANCEO CAMIONETA,106,SERVICIOS,4,31034.0,Engativa,Calle 80,31
2016-01-09,C800214037,EXPRESER SAS,5,REB-0063,REENC XZE1 235/75R17.5,102,REENCAUCHE,1,29892.0,Engativa,Ventas externas,22
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-02,C17050457,GAMBOA PAEZ JOSE MARIO,44,SE-CS-1-0001,ALINEACION AUTO,106,SERVICIOS,1,37815.0,Engativa,Calle 80,31
2020-12-02,C51590721,DAZA MIRIAM,44,SE-CS-1-0006,ALINEACION BUS,106,SERVICIOS,1,37815.0,Engativa,Calle 80,31
2020-12-02,C860517112,TRANSPORTES ESPECIALES COLEGIOS Y TURISMO SA,50,SE-CS-1-0005,ALINEACION BUSETA O CAMION LIVIANO,106,SERVICIOS,1,37815.0,Puente aranda,Calle 13,35
